In [11]:
import os
import cv2
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import scipy.io as sio
import plotly.io as pio
import plotly.express as px
import pandas as pd

from my_funcs.plot_functions import mask_roi_finder
from my_funcs.plot_functions import mask_check_plot
from my_funcs.plot_functions import compare_txt_method

from my_funcs.cest_functions import bruker_dataset_creator
from my_funcs.cest_functions import dicom_data_arranger
from my_funcs.cest_functions import z_spec_rearranger
from my_funcs.cest_functions import offset_rearranger

from my_funcs.b0_mapping_functions import wassr_b0_mapping
from my_funcs.b0_mapping_functions import b0_correction


## 1.1. Enter Data: ##

In [12]:
phantom_choice = 2

# Root stats:
general_fn = os.path.abspath(os.curdir)
# Subject stats:
glu_phantom_fns = [os.path.join(general_fn, 'scans', 'data_glutamate_phantoms_jan16_2024',
                                '20240116_091014_OrPerlman_phantom_glu_10_20_30mM_pH7_a_1_1'),
                   os.path.join(general_fn, 'scans', 'data_glutamate_phantoms_jan16_2024',
                                '20240116_114710_OrPerlman_phantom_glu_5_10_15mM_pH7_a_1_1'),
                   os.path.join(general_fn, 'scans', 'data_glutamate_phantoms_jan16_2024',
                                '20240116_132701_OrPerlman_phantom_glu_20mM_pH6p8_7_7p2_a_1_1')]  # January
glu_phantom_fn = glu_phantom_fns[phantom_choice-1]
txt_file_name = 'labarchive_notes.txt'


In [13]:
_, files_fn, bruker_dataset = bruker_dataset_creator(glu_phantom_fn, txt_file_name, 'EPI_4uT')
_, _, bruker_dataset_mask = bruker_dataset_creator(glu_phantom_fn, txt_file_name, '107a')  # always takes mask from 107a
e_r = 0
vial_rois, full_mask, _ = mask_roi_finder(bruker_dataset_mask, e_r)
    
dict_fn = os.path.join(files_fn, 'dict.mat')
quant_fn = os.path.join(files_fn, 'quant_maps.mat')
zspec_dict = sio.loadmat(dict_fn)
zspec_quant = sio.loadmat(quant_fn)

t1w = zspec_dict['t1w'][0]
t2w = zspec_dict['t2w'][0]
fs = zspec_dict['fs_0'][0]
ksw = zspec_dict['ksw_0'][0]
sig = zspec_dict['sig']
m_id = zspec_quant['match_id']
vial_20_m = m_id[46,31] # phantom 1
# vial_10_m = m_id[28,47] # phantom 1
vial_10_m = m_id[27,19] # phantom 2

py_df = pd.DataFrame({
        't1w': t1w,
        't2w': t2w,
        'fs': fs,
        'ksw': ksw,
    })
py_df_sig = pd.DataFrame(sig)
# Concatenate the existing DataFrame with the new columns
py_df = pd.concat([py_df, py_df_sig], axis=1)
py_df_sorted = py_df.sort_values(by=['t1w', 't2w', 'fs', 'ksw'])

n = int(vial_20_m)
# n = 19
py_sig = offset_rearranger(py_df_sorted.iloc[n, 4:].values)

display(py_df_sorted)
# fig = px.imshow(zspec_quant['ksw'] * full_mask)
# fig.show()
# 
# fig = px.imshow(zspec_quant['t1w'] * full_mask)
# fig.show()
# 
# fig = px.imshow(zspec_quant['t2w'] * full_mask)
# fig.show()
# 
# fig = px.imshow(zspec_quant['match_id'] * full_mask)
# fig.show()

py_df_sorted.iloc[n, :]

,t1w,t2w,fs,ksw,0,1,2,3,4,5,...,48,49,50,51,52,53,54,55,56,57
0,2.8,1.00,0.000273,5000,0.998382,0.892665,0.899036,0.891114,0.898148,0.893126,...,0.544237,0.413460,0.501447,0.251029,0.296355,0.195765,0.226270,0.044323,0.022626,0.027426
1,2.8,1.00,0.000273,5100,0.998380,0.892595,0.898984,0.891041,0.898095,0.893056,...,0.543609,0.414142,0.500725,0.251230,0.295785,0.195823,0.225832,0.044519,0.022579,0.027492
2,2.8,1.00,0.000273,5200,0.998379,0.892532,0.898938,0.890977,0.898048,0.892996,...,0.543013,0.414860,0.500046,0.251463,0.295255,0.195914,0.225432,0.044723,0.022546,0.027565
3,2.8,1.00,0.000273,5300,0.998377,0.892478,0.898898,0.890921,0.898006,0.892944,...,0.542447,0.415611,0.499409,0.251727,0.294763,0.196037,0.225068,0.044933,0.022525,0.027644
4,2.8,1.00,0.000273,5400,0.998376,0.892430,0.898863,0.890872,0.897970,0.892901,...,0.541912,0.416391,0.498813,0.252019,0.294309,0.196189,0.224738,0.045150,0.022516,0.027729
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7483,4.5,1.75,0.000273,7100,0.998792,0.795418,0.804898,0.797425,0.808061,0.803424,...,0.485537,0.383189,0.461453,0.237631,0.272391,0.183831,0.207404,0.064793,0.015515,0.054405
7484,4.5,1.75,0.000273,7200,0.998794,0.795453,0.804921,0.797457,0.808088,0.803469,...,0.485355,0.384266,0.461367,0.238185,0.272412,0.184366,0.207515,0.065213,0.016105,0.054684
7485,4.5,1.75,0.000273,7300,0.998795,0.795491,0.804946,0.797493,0.808117,0.803518,...,0.485191,0.385344,0.461305,0.238745,0.272457,0.184914,0.207644,0.065637,0.016689,0.054967
7486,4.5,1.75,0.000273,7400,0.998797,0.795531,0.804972,0.797531,0.808148,0.803568,...,0.485044,0.386421,0.461266,0.239311,0.272523,0.185472,0.207790,0.066065,0.017267,0.055254


t1w       2.800000
t2w       1.750000
fs        0.000273
ksw    7500.000000
0         0.998837
          ...     
53        0.264769
54        0.295794
55        0.082284
56        0.025857
57        0.057562
Name: 415, Length: 62, dtype: float64

## 1.2. scan data ##

In [14]:
conc_ls = [['30 mM', '20 mM', '10 mM'], ['10 mM', '5 mM', '15 mM'], ['20 mM', '20 mM', '20 mM']]  # this is the order of the masks using 107a
ph_ls = [['pH 7', 'pH 7', 'pH 7'], ['pH 7', 'pH 7', 'pH 7'], ['pH 7', 'pH 7.2', 'pH 6.8']]  # this is the order of the masks using 107a

tags = [['a', 'b', 'c'], ['b', 'c', 'a'], ['b', 'a', 'c']]

tag_x_locs = [[-3,3,-5], [-2,3,-2], [3, 5, -2]]
tag_y_locs = [[-13,13,12], [-14,13,13], [-13, 13, 13]]

cest_prtcls_names = [['cest_EPI_3p6uT', 'cest_EPI_4uT', 'cest_EPI_6uT'],
                     ['cest_EPI_3p6uT', 'cest_EPI_4uT', 'cest_EPI_6uT'],
                     ['cest_EPI_3p6uT', 'cest_EPI_4uT', 'cest_EPI_6uT']]

B_names = [['3p6uT', '4uT', '6uT'],['3p6uT', '4uT', '6uT'], ['3p6uT', '4uT', '6uT']]


## 1. z-spectra ##

In [15]:
mean_z_spectra = []
mean_z_spectra_n_c = []
gyro_ratio_hz = 42.5764  # for H [Hz/uT]
b0 = 7

for phantom_i in [0, 1, 2]:  # [0, 1, 2]
    cest_prtcl_names = cest_prtcls_names[phantom_i]
    glu_phantom_fn = glu_phantom_fns[phantom_i]
    B_name = B_names[phantom_i]
    
    # mask
    _, _, bruker_dataset_mask = bruker_dataset_creator(glu_phantom_fn, txt_file_name, '107a')  # always takes mask from 107a
    vial_rois, full_mask, _ = mask_roi_finder(bruker_dataset_mask, e_r)
    
    # WASSR image
    wassr_dicom_fn, wassr_mrf_files_fn, wassr_bruker_dataset = bruker_dataset_creator(glu_phantom_fn, txt_file_name, 'WASSR')
    wassr_data = dicom_data_arranger(wassr_bruker_dataset, wassr_dicom_fn)
    M0_wassr, arr_wassr_spec = z_spec_rearranger(wassr_data)  # (21, 64, 64)

    wassr_norm = np.divide(arr_wassr_spec, np.where(M0_wassr == 0, 1e-8, M0_wassr))  # (22, 64, 64) full_mask
    offset_hz = offset_rearranger(wassr_bruker_dataset['SatFreqList'].value)
    offset_ppm = offset_hz / (gyro_ratio_hz * b0)
    b0_map = wassr_b0_mapping(wassr_norm, full_mask, w_x=offset_ppm, MainFieldMHz=gyro_ratio_hz*b0)
    
    mean_z_spectrum = np.zeros([3, len(cest_prtcl_names), 57])
    mean_z_spectrum_not_cor = np.zeros([3, len(cest_prtcl_names), 57])
    for cest_prtcl_i, cest_prtcl_name in enumerate(cest_prtcl_names):  # 0.7, 2, 4, 6
        cur_B_name = B_name[cest_prtcl_i]

        # z-spec
        glu_phantom_dicom_fn, glu_phantom_mrf_files_fn, bruker_dataset = bruker_dataset_creator(glu_phantom_fn, txt_file_name, cest_prtcl_name)  # (58/52, 64, 64) [M0,7,-7,6.75,-6.75,...,0.25,-0.25,0]
        cest_data = dicom_data_arranger(bruker_dataset, glu_phantom_dicom_fn)
        # cest_data_0 = cest_data[0, :, :] * full_mask
        # cest_data_1 = cest_data[1, :, :] * full_mask
        # cest_data_div = cest_data_1 / cest_data_0
        M0_cest, arr_z_spec = z_spec_rearranger(cest_data)
        z_spec_norm = np.divide(arr_z_spec, np.where(M0_cest == 0, 1e-8, M0_cest))  # (51/57, 64, 64) full_mask
        
        # offset vector
        offset_hz = offset_rearranger(bruker_dataset['SatFreqList'].value)
        offset_ppm = offset_hz / (gyro_ratio_hz * b0)
        # offset_ppm = np.round((offset_ppm) / 0.05) * 0.05
        # offset_hz = offset_ppm[1:] * gyro_ratio_hz * b0
    
        b0_cor_zspec = b0_correction(b0_map, z_spec_norm, offset_hz) # have not checked!

        for vial_i, vial_roi in enumerate(vial_rois):
            roi_mask = np.zeros_like(full_mask)
            rr, cc = vial_roi.coords[:, 0], vial_roi.coords[:, 1]
            roi_mask[rr, cc] = 1
            # roi_mask = cv2.cvtColor(roi_mask, cv2.COLOR_BGR2GRAY)
            roi_mask = roi_mask.astype(np.uint8)
            
            for c_i in range(z_spec_norm.shape[0]):
                [[mean_vial]], _ = cv2.meanStdDev(z_spec_norm[c_i, :, :], mask=roi_mask)  # before b0 correction
                mean_z_spectrum_not_cor[vial_i,cest_prtcl_i,c_i] = mean_vial
                [[mean_vial]], _ = cv2.meanStdDev(b0_cor_zspec[c_i, :, :], mask=roi_mask)
                mean_z_spectrum[vial_i,cest_prtcl_i,c_i] = mean_vial

    mean_z_spectra.append(mean_z_spectrum)
    mean_z_spectra_n_c.append(mean_z_spectrum_not_cor)


WASSR B0 mapping took 3.355 seconds
WASSR B0 mapping took 4.005 seconds
WASSR B0 mapping took 3.443 seconds


In [16]:
l1, l2, l3 = 5, 7, 7
w1 = l1 / (l1+l2+l3)
w2 = l2 / (l1+l2+l3)
w3 = l3 / (l1+l2+l3)


In [17]:
for phantom_i in [0, 1, 2]:
    cest_prtcl_names = cest_prtcls_names[phantom_i]  # 3.6, 4, 6
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c']
    conc_l = conc_ls[phantom_i]
    ph_l = ph_ls[phantom_i]
    tag = tags[phantom_i]
    abc_id = [tag.index('a'), tag.index('b'), tag.index('c')]

    fig = make_subplots(rows=1, cols=len(cest_prtcl_names),
                        subplot_titles=B_names[phantom_i], vertical_spacing = 0.07, horizontal_spacing=0.01, column_widths=[w1, w2, w3])
    fig_2 = make_subplots(rows=1, cols=len(cest_prtcl_names),
                          subplot_titles=B_names[phantom_i], vertical_spacing = 0.07, horizontal_spacing=0.05, column_widths=[w1, w2, w3])

    mean_z_spectrum = mean_z_spectra[phantom_i]
    mean_z_spectrum_not_cor = mean_z_spectra_n_c[phantom_i]
    # Iterate over each channel
    for B_idx in range(len(cest_prtcl_names)):
        # Iterate over each image
        for tag_i, cur_tag in enumerate(['a', 'b', 'c']):
            vial_i = tag.index(cur_tag)
            z_vial_B = mean_z_spectrum[vial_i,B_idx,:]
            z_vial_B_not_cor = mean_z_spectrum_not_cor[vial_i,B_idx,:]
            
            if B_idx == 0:
                ppm_lim = 5
                z_vial_B = z_vial_B[:51]
            else:
                ppm_lim = 7

            # MTR asym calculation
            cha_n = len(z_vial_B)
            mid_i = int(cha_n/2 - 1)
            last_i = int(cha_n-1)
            positives = z_vial_B[0:(mid_i+2)]  # positives to 0 (including)
            negatives = z_vial_B[last_i:mid_i:-1]  # negatives to 0 (including)

            MTR_asym = negatives-positives
            ppm_asym = np.linspace(ppm_lim, 0, num=(mid_i+2))     

            ppm = np.linspace(ppm_lim, -ppm_lim, num=cha_n)

            col_idx = B_idx + 1
            fig.add_trace(go.Scatter(x=ppm, y=z_vial_B, line=dict(color=colors[tag_i]), name=f'{cur_tag} {conc_l[vial_i]}, {ph_l[vial_i]}', legendgroup=f'{1}_{col_idx}_1'), row=1, col=col_idx)
            if B_idx==1:
                fig.add_trace(go.Scatter(x=ppm, y=py_sig, line=dict(color=colors[tag_i]), name=f'just_try', legendgroup=f'{1}_{col_idx}_1'), row=1, col=col_idx)
            # fig.add_trace(go.Scatter(x=ppm, y=z_vial_B_not_cor, line=dict(color='gray'), name=f'{cur_tag} {conc_l[vial_i]}, {ph_l[vial_i]}', legendgroup=f'{1}_{col_idx}_1'), row=1, col=col_idx)  # later delete!
            
            fig.add_trace(go.Scatter(x=ppm_asym, y=MTR_asym, line=dict(color=colors[tag_i]), name=f'{conc_l[vial_i]}, {ph_l[vial_i]}', legendgroup=f'{1}_{col_idx}_2'), row=1, col=col_idx)
            fig.update_xaxes(title_text='ppm', row=1, col=col_idx)
            
            fig_2.add_trace(go.Scatter(x=ppm_asym, y=MTR_asym, line=dict(color=colors[tag_i]), name=f'{conc_l[vial_i]}, {ph_l[vial_i]}', legendgroup=f'{1}_{col_idx}_2'), row=1, col=col_idx)
            fig_2.update_xaxes(title_text='ppm', row=1, col=col_idx)
            
    fig.update_yaxes(title_text='$M_{sat}/M_0$', row=1, col=1, title_standoff=2)
    fig_2.update_yaxes(title_text='MTR-asym', row=1, col=1)
    
    # Update layout
    fig.update_layout(template='plotly_white',  # Set the theme to plotly white
                      title_text=f'Phantom {phantom_i+1} Z-spectrum',
                      height=250, width=1000,
                      title=dict(x=0.02, y=0.97),
                      margin=dict(l=45, r=0, t=50, b=0)
                      )  # Adjust the title position
    
    fig_2.update_layout(template='plotly_white',  # Set the theme to plotly white
                  title_text=f'Phantom {phantom_i+1} MTR-asym',
                  height=250, width=800,
                  title=dict(x=0.02, y=0.97),
                  margin=dict(l=0, r=0, t=60, b=0)
                        )  # Adjust the title position

    # Set axes for z-spectrum
    fig.update_xaxes(autorange='reversed',tickmode='linear', tick0=0, dtick=1)
    fig.update_yaxes(tickmode='linear', tick0=0, dtick=0.2, range=[-0.1, 1])
    
    # # Set axes for MTR-asym
    fig_2.update_xaxes(autorange='reversed', dtick=1)
    # fig_2.update_xaxes(tickvals=list(range(5)), ticktext=list(map(str, range(4, -1, -1))))
    fig_2.update_yaxes(tickmode='linear', tick0=0, dtick=0.05, range=[0, 0.25], tickformat='.0%')    

    # Show only specific legend groups
    for trace in fig.data:
        trace.showlegend = (trace.legendgroup == '1_1_1')
        
    # Show only specific legend groups
    for trace in fig_2.data:
        trace.showlegend = (trace.legendgroup == '1_1_2')

    # Show the plot
    print(f'Phantom {phantom_i+1}: Glutamate (a) {conc_l[abc_id[0]]}, {ph_l[abc_id[0]]} '
                                                       f'(b) {conc_l[abc_id[1]]}, {ph_l[abc_id[1]]} '
                                                       f'(c) {conc_l[abc_id[2]]}, {ph_l[abc_id[2]]}')
    fig.show()
    fig_2.show()
    
    pio.write_image(fig, f'images/z_spec/zspec_phantom_{phantom_i+1}.jpeg')
    pio.write_image(fig_2, f'images/z_spec/mtr_phantom_{phantom_i+1}.jpeg')


Phantom 1: Glutamate (a) 30 mM, pH 7 (b) 20 mM, pH 7 (c) 10 mM, pH 7


Phantom 2: Glutamate (a) 15 mM, pH 7 (b) 10 mM, pH 7 (c) 5 mM, pH 7


Phantom 3: Glutamate (a) 20 mM, pH 7.2 (b) 20 mM, pH 7 (c) 20 mM, pH 6.8


# very strange, results from the same vial are very different! #

# 10 mM, 7 pH #

In [18]:
fig = make_subplots(rows=1, cols=3,
                    subplot_titles=B_names[phantom_i], vertical_spacing = 0.07, horizontal_spacing=0.01, column_widths=[w1, w2, w3])
fig_2 = make_subplots(rows=1, cols=3,
                      subplot_titles=B_names[phantom_i], vertical_spacing = 0.07, horizontal_spacing=0.05, column_widths=[w1, w2, w3])

for phantom_i in [0, 1, 2]:
    cest_prtcl_names = cest_prtcls_names[phantom_i]  # 3.6, 4, 6
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c']
    conc_l = conc_ls[phantom_i]
    ph_l = ph_ls[phantom_i]
    tag = tags[phantom_i]
    abc_id = [tag.index('a'), tag.index('b'), tag.index('c')]

    mean_z_spectrum = mean_z_spectra[phantom_i]
    mean_z_spectrum_not_cor = mean_z_spectra_n_c[phantom_i]
    # Iterate over each channel
    for B_idx in range(len(cest_prtcl_names)):
        # Iterate over each image
        for tag_i, cur_tag in enumerate(['a', 'b', 'c']):
            if (phantom_i == 0) & (cur_tag == 'c'):
                plot_flag = 1
            elif (phantom_i == 1) & (cur_tag == 'b'):
                plot_flag = 1
            else:
                plot_flag = 0

            if plot_flag:
                vial_i = tag.index(cur_tag)
                z_vial_B = mean_z_spectrum[vial_i,B_idx,:]
                z_vial_B_not_cor = mean_z_spectrum_not_cor[vial_i,B_idx,:]
                if B_idx == 0:
                    ppm_lim = 5
                    z_vial_B = z_vial_B[:51]
                else:
                    ppm_lim = 7
                
                # MTR asym calculation
                cha_n = len(z_vial_B)
                mid_i = int(cha_n/2 - 1)
                last_i = int(cha_n-1)
                positives = z_vial_B[0:(mid_i+2)]  # positives to 0 (including)
                negatives = z_vial_B[last_i:mid_i:-1]  # negatives to 0 (including)
    
                MTR_asym = negatives-positives
                ppm_asym = np.linspace(ppm_lim, 0, num=(mid_i+2))     
    
                ppm = np.linspace(ppm_lim, -ppm_lim, num=cha_n)
    
                col_idx = B_idx + 1
                fig.add_trace(go.Scatter(x=ppm, y=z_vial_B, line=dict(color=colors[tag_i]), name=f'phantom {phantom_i+1}, {conc_l[vial_i]}, {ph_l[vial_i]}', legendgroup=f'{1}_{col_idx}_1'), row=1, col=col_idx)
                # fig.add_trace(go.Scatter(x=ppm, y=z_vial_B_not_cor, line=dict(color='gray'), name=f'{cur_tag} {conc_l[vial_i]}, {ph_l[vial_i]}', legendgroup=f'{1}_{col_idx}_1'), row=1, col=col_idx)  # later delete!
                fig.add_trace(go.Scatter(x=ppm_asym, y=MTR_asym, line=dict(color=colors[tag_i]), name=f'phantom {phantom_i+1}, {conc_l[vial_i]}, {ph_l[vial_i]}', legendgroup=f'{1}_{col_idx}_2'), row=1, col=col_idx)
                fig.update_xaxes(title_text='ppm', row=1, col=col_idx)
                
                fig_2.add_trace(go.Scatter(x=ppm_asym, y=MTR_asym, line=dict(color=colors[tag_i]), name=f'phantom {phantom_i+1}, {conc_l[vial_i]}, {ph_l[vial_i]}', legendgroup=f'{1}_{col_idx}_2'), row=1, col=col_idx)
                fig_2.update_xaxes(title_text='ppm', row=1, col=col_idx)
            
fig.update_yaxes(title_text='$M_{sat}/M_0$', row=1, col=1, title_standoff=2)
fig_2.update_yaxes(title_text='MTR-asym', row=1, col=1)

# Update layout
fig.update_layout(template='plotly_white',  # Set the theme to plotly white
                  title_text=f'Glutamate 10 mM, pH 7 Z-spectrum',
                  height=250, width=1000,
                  title=dict(x=0.02, y=0.97),
                  margin=dict(l=45, r=0, t=50, b=0)
                  )  # Adjust the title position

fig_2.update_layout(template='plotly_white',  # Set the theme to plotly white
              title_text=f'Glutamate 10 mM, pH 7 MTR-asym',
              height=250, width=800,
              title=dict(x=0.02, y=0.97),
              margin=dict(l=0, r=0, t=60, b=0)
                    )  # Adjust the title position

# Set axes for z-spectrum
fig.update_xaxes(autorange='reversed',tickmode='linear', tick0=0, dtick=1)
fig.update_yaxes(tickmode='linear', tick0=0, dtick=0.2, range=[-0.1, 1])

# # Set axes for MTR-asym
fig_2.update_xaxes(autorange='reversed', dtick=1)
# fig_2.update_xaxes(tickvals=list(range(5)), ticktext=list(map(str, range(4, -1, -1))))
fig_2.update_yaxes(tickmode='linear', tick0=0, dtick=0.05, range=[0, 0.25], tickformat='.0%')    

# Show only specific legend groups
for trace in fig.data:
    trace.showlegend = (trace.legendgroup == '1_1_1')
    
# Show only specific legend groups
for trace in fig_2.data:
    trace.showlegend = (trace.legendgroup == '1_1_2')

# Show the plot
print(f'Glutamate 10 mM, pH 7')
fig.show()
fig_2.show()

pio.write_image(fig, f'images/z_spec/zspec_10_cmpr.jpeg')
pio.write_image(fig_2, f'images/z_spec/mtr_10_cmpr.jpeg')


Glutamate 10 mM, pH 7


In [19]:
fig = make_subplots(rows=1, cols=3,
                    subplot_titles=B_names[phantom_i], vertical_spacing = 0.07, horizontal_spacing=0.01, column_widths=[w1, w2, w3])
fig_2 = make_subplots(rows=1, cols=3,
                      subplot_titles=B_names[phantom_i], vertical_spacing = 0.07, horizontal_spacing=0.05, column_widths=[w1, w2, w3])

for phantom_i in [0, 1, 2]:
    cest_prtcl_names = cest_prtcls_names[phantom_i]  # 3.6, 4, 6
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c']
    conc_l = conc_ls[phantom_i]
    ph_l = ph_ls[phantom_i]
    tag = tags[phantom_i]
    abc_id = [tag.index('a'), tag.index('b'), tag.index('c')]

    mean_z_spectrum = mean_z_spectra[phantom_i]
    mean_z_spectrum_not_cor = mean_z_spectra_n_c[phantom_i]
    # Iterate over each channel
    for B_idx in range(len(cest_prtcl_names)):
        # Iterate over each image
        for tag_i, cur_tag in enumerate(['a', 'b', 'c']):
            if (phantom_i == 0) & (cur_tag == 'b'):
                plot_flag = 1
            elif (phantom_i == 2) & (cur_tag == 'b'):
                plot_flag = 1
                tag_c = 1
            else:
                plot_flag = 0
                tag_c = 2

            if plot_flag:
                vial_i = tag.index(cur_tag)
                z_vial_B = mean_z_spectrum[vial_i,B_idx,:]
                z_vial_B_not_cor = mean_z_spectrum_not_cor[vial_i,B_idx,:]
                if B_idx == 0:
                    ppm_lim = 5
                    z_vial_B = z_vial_B[:51]
                else:
                    ppm_lim = 7
                
                # MTR asym calculation
                cha_n = len(z_vial_B)
                mid_i = int(cha_n/2 - 1)
                last_i = int(cha_n-1)
                positives = z_vial_B[0:(mid_i+2)]  # positives to 0 (including)
                negatives = z_vial_B[last_i:mid_i:-1]  # negatives to 0 (including)
    
                MTR_asym = negatives-positives
                ppm_asym = np.linspace(ppm_lim, 0, num=(mid_i+2))     
    
                ppm = np.linspace(ppm_lim, -ppm_lim, num=cha_n)
    
                col_idx = B_idx + 1
                fig.add_trace(go.Scatter(x=ppm, y=z_vial_B, line=dict(color=colors[tag_c]), name=f'phantom {phantom_i+1}, {conc_l[vial_i]}, {ph_l[vial_i]}', legendgroup=f'{1}_{col_idx}_1'), row=1, col=col_idx)
                # fig.add_trace(go.Scatter(x=ppm, y=z_vial_B_not_cor, line=dict(color='gray'), name=f'{cur_tag} {conc_l[vial_i]}, {ph_l[vial_i]}', legendgroup=f'{1}_{col_idx}_1'), row=1, col=col_idx)  # later delete!
                fig.add_trace(go.Scatter(x=ppm_asym, y=MTR_asym, line=dict(color=colors[tag_c]), name=f'phantom {phantom_i+1}, {conc_l[vial_i]}, {ph_l[vial_i]}', legendgroup=f'{1}_{col_idx}_2'), row=1, col=col_idx)
                fig.update_xaxes(title_text='ppm', row=1, col=col_idx)
                
                fig_2.add_trace(go.Scatter(x=ppm_asym, y=MTR_asym, line=dict(color=colors[tag_c]), name=f'phantom {phantom_i+1}, {conc_l[vial_i]}, {ph_l[vial_i]}', legendgroup=f'{1}_{col_idx}_2'), row=1, col=col_idx)
                fig_2.update_xaxes(title_text='ppm', row=1, col=col_idx)
            
fig.update_yaxes(title_text='$M_{sat}/M_0$', row=1, col=1, title_standoff=2)
fig_2.update_yaxes(title_text='MTR-asym', row=1, col=1)

# Update layout
fig.update_layout(template='plotly_white',  # Set the theme to plotly white
                  title_text=f'Glutamate 20 mM, pH 7 Z-spectrum',
                  height=250, width=1000,
                  title=dict(x=0.02, y=0.97),
                  margin=dict(l=45, r=0, t=50, b=0)
                  )  # Adjust the title position

fig_2.update_layout(template='plotly_white',  # Set the theme to plotly white
              title_text=f'Glutamate 20 mM, pH 7 MTR-asym',
              height=250, width=800,
              title=dict(x=0.02, y=0.97),
              margin=dict(l=0, r=0, t=60, b=0)
                    )  # Adjust the title position

# Set axes for z-spectrum
fig.update_xaxes(autorange='reversed',tickmode='linear', tick0=0, dtick=1)
fig.update_yaxes(tickmode='linear', tick0=0, dtick=0.2, range=[-0.1, 1])

# # Set axes for MTR-asym
fig_2.update_xaxes(autorange='reversed', dtick=1)
# fig_2.update_xaxes(tickvals=list(range(5)), ticktext=list(map(str, range(4, -1, -1))))
fig_2.update_yaxes(tickmode='linear', tick0=0, dtick=0.05, range=[0, 0.25], tickformat='.0%')    

# Show only specific legend groups
for trace in fig.data:
    trace.showlegend = (trace.legendgroup == '1_1_1')
    
# Show only specific legend groups
for trace in fig_2.data:
    trace.showlegend = (trace.legendgroup == '1_1_2')

# Show the plot
print(f'Glutamate 20 mM, pH 7')
fig.show()
fig_2.show()

pio.write_image(fig, f'images/z_spec/zspec_20_cmpr.jpeg')
pio.write_image(fig_2, f'images/z_spec/mtr_20_cmpr.jpeg')


Glutamate 20 mM, pH 7


In [20]:
# Create traces
trace1 = go.Scatter(x=np.arange(len(py_sig)), y=py_sig, mode='lines', name='py_sig')


# Create layout
layout = go.Layout(title='Comparison of py_sig and mtlb_sig',
                   xaxis=dict(title='X-axis'),
                   yaxis=dict(title='Y-axis'))

# Create figure
fig = go.Figure(data=[trace1], layout=layout)

# Display the Plotly figure
display(fig)
